# You're Toxic, I'm Slippin' Under: Toxic Comment Classification Challenge

#### STINTSY S13 Group 8
- VICENTE, Francheska Josefa
- VISTA, Sophia Danielle S.

## Requirements and Imports
Before starting, the relevant libraries and files in building and training the model should be loaded into the notebook first.

### Import
Several libraries are required to perform a thorough analysis of the dataset. Each of these libraries will be imported and described below:

#### Basic Libraries 
Import `numpy` and `pandas`.
- `numpy` contains a large collection of mathematical functions
- `pandas` contains functions that are designed for data manipulation and data analysis

In [2]:
import numpy as np
import pandas as pd

#### Natural Language Processing Libraries 
- `re` is a module that allows the use of regular expressions
- `nltk` provides functions for processing text data
- `stopwords` is a corpus from NLTK, which includes a compiled list of stopwords
- `Counter` is from Python's `collections` module, which is helpful for tokenization
- `string` contains functions for string operations

In [4]:
import sys
!{sys.executable} -m pip install scikit-multilearn

In [4]:
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#### Machine Learning Libraries

In [5]:
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier

import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [5]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

### Datasets and Files


In [6]:
train = pd.read_csv('cleaned_data/cleaned_train.csv')
test = pd.read_csv('cleaned_data/cleaned_test.csv')

## Trying different Models

In [7]:
test ['comment_text'] = test ['comment_text'].apply(lambda x: np.str_(x))
train ['comment_text'] = train ['comment_text'].apply(lambda x: np.str_(x))

In [8]:
X_train = train ['comment_text']
y_train = train.loc [ : , 'toxic' : ]

X_test = test ['comment_text']

In [9]:
classes = train.columns [2:]

In [10]:
class mn_hyper_parameter:
    def __init__(self, class_, alpha, fit_prior):
        self.class_ = class_
        self.alpha = alpha
        self.fit_prior = fit_prior

In [11]:
class lr_hyperparameter:
    def __init__(self, class_, c, max_iter):
        self.class_ = class_
        self.c = c
        self.max_iter = max_iter

#### Helper Functions

In [12]:
def compute_accuracy(predictions, actual):
    accuracy = np.sum (predictions == actual) / len (predictions) * 100
    return accuracy

In [13]:
def to_submission_csv(predictions, filename):
    sample_submission = pd.read_csv('data/sample_submission.csv')
    sample_submission ['id'] = test ['id'] 
    counter = 0

    for i in range (6):
        sample_submission[classes [i]] = predictions[:, i : i + 1]

    sample_submission.to_csv(f'results/' + filename + '.csv', index = False) 

In [14]:
def to_submission_csv_multiclass (predictions, filename):
    sample_submission = pd.read_csv('data/sample_submission.csv')
    sample_submission ['id'] = test ['id'] 
    counter = 0

    for i in range (6):
        temp = list(zip(*predictions[i]))
        sample_submission[classes [i]] = temp[1]

    sample_submission.to_csv(f'results/' + filename + '.csv', index = False) 

In [21]:
def train_models(model):
    predictions_count = np.zeros((len(test), len(classes)))
    predictions_tfidf = np.zeros((len(test), len(classes)))
    models = []
    
    for i in range(6):
        print('Fitting', classes[i] + '...')

        mdl = model
        mdl.fit(count_train, y_train[classes[i]])
        print('Count Vectors:', compute_accuracy(mdl.predict(count_train), y_train[classes[i]]))
        predictions_count[:,i] = mdl.predict_proba(count_test)[:,1]

        mdl = model
        mdl.fit(tfidf_train, y_train[classes[i]])
        print('TF-IDF Vectors:', compute_accuracy(mdl.predict(tfidf_train), y_train[classes[i]]))
        predictions_tfidf[:,i] = mdl.predict_proba(tfidf_test)[:,1]
    
    return models, predictions_count, predictions_tfidf

In [22]:
def tune_and_train_models(model):
    predictions_count_tuned = np.zeros((len(test), len(classes)))
    predictions_tfidf_tuned = np.zeros((len(test), len(classes)))

    for i in range(6):
        print('Fitting', classes[i] + '...')

        mdl_tuned = model
        mdl_tuned.fit(count_train, y_train[classes[i]])
        print('Count Vectors:', compute_accuracy(mdl_tuned.predict(count_train), y_train[classes[i]]), mdl_tuned.best_params_)
        predictions_count_tuned[:,i] = mdl_tuned.predict_proba(count_test)[:,1]

        mdl_tuned = model
        mdl_tuned.fit(tfidf_train, y_train[classes[i]])
        print('TF-IDF Vectors:', compute_accuracy(mdl_tuned.predict(tfidf_train), y_train[classes[i]]), mdl_tuned.best_params_)
        predictions_tfidf_tuned[:,i] = mdl_tuned.predict_proba(tfidf_test)[:,1]
    
    return predictions_count_tuned, predictions_tfidf_tuned

### TF-IDF Vectorizer

In [15]:
tfidf_vectorizer = TfidfVectorizer()

In [16]:
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

In [17]:
tfidf_test = tfidf_vectorizer.transform(X_test)

In [18]:
tfidf_vectorizer_5000 = TfidfVectorizer(max_features = 5000)

In [19]:
tfidf_train_5000 = tfidf_vectorizer_5000.fit_transform(X_train)

In [20]:
tfidf_test_5000 = tfidf_vectorizer_5000.transform(X_test)

In [21]:
count_vectorizer = CountVectorizer()

In [22]:
count_train = count_vectorizer.fit_transform(X_train)

In [23]:
count_test = count_vectorizer.transform(X_test)

In [24]:
count_vectorizer_5000 = CountVectorizer(max_features = 5000)

In [25]:
count_train_5000 = count_vectorizer_5000.fit_transform(X_train)

In [26]:
count_tes_5000t = count_vectorizer_5000.transform(X_test)

In [27]:
parameters_mn_multi = [
    {
        'classifier': [MultinomialNB()],
        'classifier__alpha': [0.5, 0.6, 0.7, 0.8, 1.0],
        'classifier__fit_prior': [True, False]
    }
]

In [28]:
parameters_lr_multi = [
    {
        'classifier': [LogisticRegression()],
        'classifier__C': [1, 12, 15],
        'classifier__max_iter': [600, 1800, 3000]
    }
]

In [29]:
parameters_mn_mo = [
    {
        'estimator__alpha': [0.5, 0.6, 0.7, 0.8, 1.0],
        'estimator__fit_prior': [True, False]
    }
]

In [30]:
parameters_lr_mo = [
    {
        'estimator__C': [1, 12, 15],
        'estimator__max_iter': [600, 1800, 3000],
        'estimator__class_weight' : ['balanced', None]
    }
]

In [31]:
parameters_mnb = [
    {
        'alpha' : [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        'fit_prior' : [False, True]
    }
]

In [32]:
parameters_lr = [
    {
        'C' : [1, 12, 15],
        'max_iter' :[600, 1800, 3000, 4200]
    }
]

In [33]:
parameters_rf = [
    {
        'n_estimators' : [500, 1000, 1500],
        'min_samples_split' : [2, 10, 20],
        'max_leaf_nodes' : [15, 20, 25],
        'min_samples_leaf' : [1, 5, 10],
    }
]

### Logistic Regression

In [25]:
lr = LogisticRegression()
lr_models, predictions_lr_count, predictions_lr_tfidf = train_models(lr)

Fitting toxic...
Count Vectors: 97.1943523572579
TF-IDF Vectors: 96.23553151888501
Fitting severe_toxic...
Count Vectors: 99.19596919239711
TF-IDF Vectors: 99.12578100030707
Fitting obscene...
Count Vectors: 98.13750618846782
TF-IDF Vectors: 97.95514222509102
Fitting threat...
Count Vectors: 99.79632890688158
TF-IDF Vectors: 99.73366087822976
Fitting insult...
Count Vectors: 97.40241021238195
TF-IDF Vectors: 97.39551672923025
Fitting identity_hate...
Count Vectors: 99.27681094935797
TF-IDF Vectors: 99.24109017302642


In [26]:
to_submission_csv(predictions_lr_count, 'submission_lr_count')
to_submission_csv(predictions_lr_tfidf, 'submission_lr_tfidf')

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-1wig{font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-baqh{text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-1wig"></th>
    <th class="tg-1wig">private</th>
    <th class="tg-1wig">public</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-baqh">submission_lr_count</td>
    <td class="tg-baqh">0.93926</td>
    <td class="tg-baqh">0.94248</td>
  </tr>
  <tr>
    <td class="tg-baqh">submission_lr_tfidf</td>
    <td class="tg-baqh">0.97391</td>
    <td class="tg-baqh">0.97376</td>
  </tr>
  <tr>
</tbody>
</table>

#### Hyperparameter Tuning

In [ ]:
lr_tuned = GridSearchCV(LogisticRegression(n_jobs=-1), parameters_lr, scoring='f1', verbose=2)
lr_models_tuned, predictions_lr_count_tuned, predictions_lr_tfidf_tuned = tune_and_train_models(lr_tuned)

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-1wig{font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-baqh{text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-1wig"></th>
    <th class="tg-1wig">private</th>
    <th class="tg-1wig">public</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-baqh">submission_lr_count_tuned</td>
    <td class="tg-baqh">N/A</td>
    <td class="tg-baqh">N/A</td>
  </tr>
  <tr>
    <td class="tg-baqh">submission_lr_tfidf_tuned</td>
    <td class="tg-baqh">N/A</td>
    <td class="tg-baqh">N/A</td>
  </tr>
  <tr>
</tbody>
</table>

### Naive Bayes: Multinomial Naive Bayes

In [27]:
mnb = MultinomialNB()
mnb_models, predictions_mnb_count, predictions_mnb_tfidf = train_models(mnb)

Fitting toxic...
Count Vectors: 95.13696097661855
TF-IDF Vectors: 92.36828747078103
Fitting severe_toxic...
Count Vectors: 98.641983819115
TF-IDF Vectors: 98.99104473870565
Fitting obscene...
Count Vectors: 96.70867513520626
TF-IDF Vectors: 95.38449968979326
Fitting threat...
Count Vectors: 99.55505699657206
TF-IDF Vectors: 99.6973134216117
Fitting insult...
Count Vectors: 96.46301646289113
TF-IDF Vectors: 95.35629907689994
Fitting identity_hate...
Count Vectors: 98.77233331871079
TF-IDF Vectors: 99.11074067343063


In [28]:
to_submission_csv(predictions_mnb_count, 'submission_mnb_count')
to_submission_csv(predictions_mnb_tfidf, 'submission_mnb_tfidf')

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-1wig{font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-baqh{text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-1wig"></th>
    <th class="tg-1wig">private</th>
    <th class="tg-1wig">public</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-baqh">submission_mnb_count</td>
    <td class="tg-baqh">0.84551</td>
    <td class="tg-baqh">0.85581</td>
  </tr>
  <tr>
    <td class="tg-baqh">submission_mnb_tfidf</td>
    <td class="tg-baqh">0.82510</td>
    <td class="tg-baqh">0.83586</td>
  </tr>
  <tr>
</tbody>
</table>

#### Hyperparameter Tuning

In [29]:
parameters_mnb = [{
    'alpha' : [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100, 1000],
    'fit_prior' : [False, True]
}]

In [30]:
mnb_tuned = GridSearchCV(MultinomialNB(), parameters_mnb, scoring='f1')
mnb_tuned_models, predictions_mnb_count_tuned, predictions_mnb_tfidf_tuned = tune_and_train_models(mnb_tuned)

Fitting toxic...
Count Vectors: 95.13696097661855 {'alpha': 1, 'fit_prior': True}
TF-IDF Vectors: 97.48262528905627 {'alpha': 0.001, 'fit_prior': True}
Fitting severe_toxic...
Count Vectors: 98.72031885492977 {'alpha': 0.001, 'fit_prior': True}
TF-IDF Vectors: 99.42157409554368 {'alpha': 0.001, 'fit_prior': True}
Fitting obscene...
Count Vectors: 96.70867513520626 {'alpha': 1, 'fit_prior': True}
TF-IDF Vectors: 98.62067668937338 {'alpha': 0.001, 'fit_prior': True}
Fitting threat...
Count Vectors: 99.31315840597603 {'alpha': 0.0001, 'fit_prior': True}
TF-IDF Vectors: 99.87403726240983 {'alpha': 1e-05, 'fit_prior': True}
Fitting insult...
Count Vectors: 96.46301646289113 {'alpha': 1, 'fit_prior': True}
TF-IDF Vectors: 98.40384531023808 {'alpha': 0.001, 'fit_prior': True}
Fitting identity_hate...
Count Vectors: 98.69337160260949 {'alpha': 0.0001, 'fit_prior': True}
TF-IDF Vectors: 99.56759060230243 {'alpha': 0.001, 'fit_prior': True}


In [31]:
to_submission_csv(predictions_mnb_count_tuned, 'submission_mnb_count_tuned')
to_submission_csv(predictions_mnb_tfidf_tuned, 'submission_mnb_tfidf_tuned')

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-1wig{font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-baqh{text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-1wig"></th>
    <th class="tg-1wig">private</th>
    <th class="tg-1wig">public</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-baqh">submission_mnb_count_tuned</td>
    <td class="tg-baqh">0.88069</td>
    <td class="tg-baqh">0.88388</td>
  </tr>
  <tr>
    <td class="tg-baqh">submission_mnb_tfidf_tuned</td>
    <td class="tg-baqh">0.91455</td>
    <td class="tg-baqh">0.91739</td>
  </tr>
  <tr>
</tbody>
</table>

### Ensemble Models: RandomForestClassifier

In [32]:
rf = RandomForestClassifier(n_jobs=-1)
rf_models, predictions_rf_count, predictions_rf_tfidf = train_models(rf)

Fitting toxic...
Count Vectors: 99.97869287025839
TF-IDF Vectors: 99.97743950968534
Fitting severe_toxic...
Count Vectors: 99.98558635341008
TF-IDF Vectors: 99.97493278853928
Fitting obscene...
Count Vectors: 99.98370631255052
TF-IDF Vectors: 99.9793195505449
Fitting threat...
Count Vectors: 99.99435987742133
TF-IDF Vectors: 99.99373319713482
Fitting insult...
Count Vectors: 99.96991934624712
TF-IDF Vectors: 99.9655325842415
Fitting identity_hate...
Count Vectors: 99.9931065168483
TF-IDF Vectors: 99.98934643512919


In [33]:
to_submission_csv(predictions_rf_count, 'submission_rf_count')
to_submission_csv(predictions_rf_tfidf, 'submission_rf_tfidf')

In [ ]:
pd.DataFrame(
    data={'private': [0.96735, 0.96784], 'public': [0.96725, 0.96710]}, 
    index=['submission_rf_count.csv', 'submission_rf_tfidf.csv']
)

#### Hyperparameter Tuning

In [36]:
parameters_rf = {
    'n_estimators' : [100, 200, 300, 400, 500],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [5, 10, 20, 30],
    'min_samples_split' : [2, 4, 6, 10, 15, 20],
    'max_leaf_nodes' : [3, 5, 10, 20, 50, 100],
}

In [39]:
rf_tuned = RandomizedSearchCV(RandomForestClassifier(n_jobs=-1), parameters_rf, scoring='f1', random_state=8, verbose=1)
rf_tuned_models, predictions_rf_count_tuned, predictions_rf_tfidf_tuned = tune_and_train_models(rf_tuned)

Fitting toxic...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Count Vectors: 90.41680505856327 {'n_estimators': 500, 'min_samples_split': 2, 'max_leaf_nodes': 50, 'max_depth': 30, 'criterion': 'entropy'}
Fitting 5 folds for each of 10 candidates, totalling 50 fits
TF-IDF Vectors: 90.4180584191363 {'n_estimators': 500, 'min_samples_split': 2, 'max_leaf_nodes': 50, 'max_depth': 30, 'criterion': 'entropy'}
Fitting severe_toxic...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Count Vectors: 99.00044494300343 {'n_estimators': 200, 'min_samples_split': 2, 'max_leaf_nodes': 20, 'max_depth': 20, 'criterion': 'gini'}
Fitting 5 folds for each of 10 candidates, totalling 50 fits
TF-IDF Vectors: 99.00044494300343 {'n_estimators': 200, 'min_samples_split': 2, 'max_leaf_nodes': 20, 'max_depth': 20, 'criterion': 'gini'}
Fitting obscene...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Count Vectors: 94.70580493949402 {'n_estimators': 500, 'min_samples_spli

In [38]:
to_submission_csv(predictions_rf_count_tuned, 'submission_rf_count_tuned')
to_submission_csv(predictions_rf_tfidf_tuned, 'submission_rf_tfidf_tuned')

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-1wig{font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-baqh{text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-1wig"></th>
    <th class="tg-1wig">private</th>
    <th class="tg-1wig">public</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-baqh">submission_rf_count_tuned</td>
    <td class="tg-baqh">0.96232</td>
    <td class="tg-baqh">0.96285</td>
  </tr>
  <tr>
    <td class="tg-baqh">submission_rf_tfidf_tuned</td>
    <td class="tg-baqh">0.95937</td>
    <td class="tg-baqh">0.95826</td>
  </tr>
  <tr>
</tbody>
</table>

### Ensemble Models: GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier()
gbc_models, predictions_gbc_count, predictions_gbc_tfidf = train_models(gbc)

In [ ]:
to_submission_csv(predictions_gbc_count, 'submission_gbc_count')
to_submission_csv(predictions_gbc_tfidf, 'submission_gbc_tfidf')

In [ ]:
pd.DataFrame(
    data={'private': [0.90663, 0.92569], 'public': [0.92024, 0.93239]}, 
    index=['submission_gbc_count.csv', 'submission_gbc_tfidf.csv']
)

#### Hyperparameter Tuning

In [ ]:
parameters_gbc = [{
    'n_estimators' : [50, 100, 250],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 1.2],
}]

In [ ]:
gbc_tuned = RandomizedSearchCV(GradientBoostingClassifier(), parameters_gbc, scoring='accuracy', random_state=8, verbose=2)
gbc_tuned_models, predictions_gbc_count_tuned, predictions_gbc_tfidf_tuned = tune_and_train_models(gbc_tuned)

In [ ]:
to_submission_csv(predictions_gbc_count_tuned, 'submission_gbc_count_tuned')
to_submission_csv(predictions_gbc_tfidf_tuned, 'submission_gbc_tfidf_tuned')

### Ensemble Models: XGBClassifier

In [ ]:
xgb = xgboost.XGBClassifier(eval_metric='logloss', verbosity=0, use_label_encoder=False)
xgb_models, predictions_xgb_count, predictions_xgb_tfidf = train_models(xgb)

In [ ]:
to_submission_csv(predictions_xgb_count, 'submission_xgb_count')
to_submission_csv(predictions_xgb_tfidf, 'submission_xgb_tfidf')

In [ ]:
parameters_xgb = [{
    'n_estimators' : [50, 100, 250],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 1.2],
}]

In [ ]:
xgb_tuned = GridSearchCV(xgboost.XGBClassifier(eval_metric='logloss', verbosity=0, use_label_encoder=False), parameters_xgb, scoring='f1')
xgb_tuned_models, predictions_xgb_count_tuned, predictions_xgb_tfidf_tuned = tune_and_train_models(xgb_tuned)

In [ ]:
to_submission_csv(predictions_xgb_count_tuned, 'submission_xgb_count_tuned')
to_submission_csv(predictions_xgb_tfidf_tuned, 'submission_xgb_tfidf_tuned')

### Ensemble Models: AdaBoostClassifier

In [ ]:
adb = AdaBoostClassifier()
adb_models, predictions_adb_count, predictions_adb_tfidf = train_models(adb)

In [ ]:
to_submission_csv(predictions_adb_count, 'submission_adb_count')
to_submission_csv(predictions_adb_tfidf, 'submission_adb_tfidf')

In [ ]:
pd.DataFrame(
    data={'private': [0.93539, 0.93830], 'public': [0.94218, 0.94145]}, 
    index=['submission_adb_count.csv', 'submission_adb_tfidf.csv']
)

In [ ]:
parameters_adb = {
    'n_estimators' : [10, 25, 50, 100, 250],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 1.2]
}

In [ ]:
adb_tuned = GridSearchCV(AdaBoostClassifier(), parameters_adb, scoring='accuracy')
adb_tuned_models, predictions_adb_count_tuned, predictions_adb_tfidf_tuned = tune_and_train_models(adb_tuned)

In [ ]:
to_submission_csv(predictions_xgb_count_tuned, 'submission_xgb_count_tuned')
to_submission_csv(predictions_xgb_tfidf_tuned, 'submission_xgb_tfidf_tuned')

### Support Vector Machines: SGDClassifier

In [ ]:
sgd = SGDClassifier(loss='log', n_jobs=-1)
sgd_models, predictions_sgd_count, predictions_sgd_tfidf = train_models(sgd)

In [ ]:
to_submission_csv(predictions_sgd_count, 'submission_sgd_count')
to_submission_csv(predictions_sgd_tfidf, 'submission_sgd_tfidf')

In [ ]:
pd.DataFrame(
    data={'private': [0.92026, 0.95112], 'public': [0.92912, 0.95232]}, 
    index=['submission_sgd_count.csv', 'submission_sgd_tfidf.csv']
)

#### Hyperparameter Tuning

In [36]:
parameters_sgd = [{
    'loss' : ['log', 'modified_huber'],
    'alpha' : [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
}]

In [37]:
sgd_tuned = GridSearchCV(SGDClassifier(n_jobs=-1), parameters_sgd, scoring='accuracy')
predictions_sgd_count_tuned, predictions_sgd_tfidf_tuned = tune_and_train_models(sgd_tuned)

Fitting toxic...
Count Vectors: 97.49139881306755 {'alpha': 1e-05, 'loss': 'log'}
TF-IDF Vectors: 98.16570680136115 {'alpha': 1e-05, 'loss': 'modified_huber'}
Fitting severe_toxic...
Count Vectors: 99.08003333939124 {'alpha': 0.01, 'loss': 'log'}
TF-IDF Vectors: 99.10760727199805 {'alpha': 1e-05, 'loss': 'log'}
Fitting obscene...
Count Vectors: 97.64681552412405 {'alpha': 1e-05, 'loss': 'log'}


KeyboardInterrupt: 

In [ ]:
to_submission_csv(predictions_rf_count_tuned, 'submission_rf_count_tuned')
to_submission_csv(predictions_rf_tfidf_tuned, 'submission_rf_tfidf_tuned')

### OneVsRest Classifier: Logistic Regression

#### Model Training

In [27]:
lr_oc_count = OneVsRestClassifier(LogisticRegression(class_weight = 'balanced', max_iter = 3000))
lr_oc_count.fit(count_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                 max_iter=3000))

In [28]:
lr_oc_tf = OneVsRestClassifier(LogisticRegression(class_weight = 'balanced', max_iter = 3000))
lr_oc_tf.fit(tfidf_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                 max_iter=3000))

In [29]:
predictions = lr_oc_tf.predict(tfidf_train)
print('TF-IDF Vectors: ' , compute_accuracy(predictions, y_train))

predictions = lr_oc_count.predict(count_train)
print('Count Vectors: ', compute_accuracy(predictions, y_train))

TF-IDF Vectors:  toxic            95.739201
severe_toxic     97.941355
obscene          98.078598
threat           99.375200
insult           96.812077
identity_hate    98.102412
dtype: float64
Count Vectors:  toxic            97.955142
severe_toxic     98.672064
obscene          98.804294
threat           99.741808
insult           97.874927
identity_hate    98.947177
dtype: float64


In [30]:
predictions_lr_oc_tf = lr_oc_tf.predict_proba(tfidf_test)
predictions_lr_oc_count = lr_oc_count.predict_proba(count_test)

In [35]:
to_submission_csv(predictions_lr_oc_tf, 'submission_oc_lr_tf')
to_submission_csv(predictions_lr_oc_count, 'submission_oc_lr_count')

In [36]:
pd.DataFrame(
    data={'private': [0.94036, 0.97558], 'public': [0.94400, 0.97621]}, 
    index=['submission_oc_lr_count.csv', 'submission_oc_lr_tf.csv']
)

,private,public
submission_oc_lr_count.csv,0.94036,0.94400
submission_oc_lr_tf.csv,0.97558,0.97621


#### Hyperparameter Tuning

In [57]:
predictions_lr_count_tuned = np.zeros((len(test), len(classes)))
predictions_lr_tfidf_tuned = np.zeros((len(test), len(classes)))

In [58]:
estimator = OneVsRestClassifier(LogisticRegression ())

In [59]:
lr_oc_tf_tuned = GridSearchCV(estimator, parameters_lr_mo, n_jobs = -1, verbose = 10, scoring = 'f1')
lr_oc_tf_tuned.fit(tfidf_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=OneVsRestClassifier(estimator=LogisticRegression()),
             n_jobs=-1,
             param_grid=[{'estimator__C': [1, 12, 15],
                          'estimator__class_weight': ['balanced', None],
                          'estimator__max_iter': [600, 1800, 3000]}],
             scoring='accuracy', verbose=10)

In [60]:
predictions = lr_oc_tf_tuned.predict(tfidf_train)
print('TF-IDF Vectors: ', compute_accuracy(predictions, y_train), lr_oc_tf_tuned.best_params_)

TF-IDF Vectors:  toxic            98.477167
severe_toxic     99.529363
obscene          99.205369
threat           99.895344
insult           98.869469
identity_hate    99.662219
dtype: float64 {'estimator__C': 12, 'estimator__class_weight': None, 'estimator__max_iter': 600}


In [61]:
predictions_lr_tfidf_tuned = lr_oc_tf_tuned.predict_proba(tfidf_test)
to_submission_csv(predictions_lr_tfidf_tuned, 'submission_oc_lr_tf_tuned')

In [ ]:
lr_oc_count_tuned = GridSearchCV(estimator, parameters_lr_mo, n_jobs = -1, verbose = 10, scoring = 'f1')
lr_oc_count_tuned.fit(count_train, y_train)   

Fitting 5 folds for each of 18 candidates, totalling 90 fits


In [ ]:
predictions = lr_oc_count_tuned.predict(count_train)
print('Count Vectors: ', compute_accuracy(predictions, y_train), lr_oc_count_tuned.best_params_)

In [ ]:
predictions_lr_count_tuned = lr_oc_count_tuned.predict_proba(count_test)
to_submission_csv(predictions_lr_count_tuned, 'submission_oc_lr_count_tuned')

In [ ]:
pd.DataFrame(
    data={'private': [0.93996, 0.97558], 'public': [0.94410, 0.97621]}, 
    index=['submission_oc_lr_count_tuned.csv', 'submission_oc_lr_tf_tuned.csv']
)

### OneVsRest Classifier: Multinomial Naive Bayes

#### Model Training

In [34]:
mn_oc_count = OneVsRestClassifier(MultinomialNB())
mn_oc_count.fit(count_train, y_train)

OneVsRestClassifier(estimator=MultinomialNB())

In [35]:
mn_oc_tf = OneVsRestClassifier(MultinomialNB())
mn_oc_tf.fit(tfidf_train, y_train)

OneVsRestClassifier(estimator=MultinomialNB())

In [36]:
predictions = mn_oc_tf.predict(tfidf_train)
print('TF-IDF Vectors: \n' , compute_accuracy(predictions, y_train))

predictions = mn_oc_count.predict(count_train)
print('Count Vectors: \n', compute_accuracy(predictions, y_train))

TF-IDF Vectors:  toxic            92.368287
severe_toxic     98.991045
obscene          95.384500
threat           99.697313
insult           95.356299
identity_hate    99.110741
dtype: float64
Count Vectors:  toxic            95.136961
severe_toxic     98.641984
obscene          96.708675
threat           99.555057
insult           96.463016
identity_hate    98.772333
dtype: float64


In [37]:
predictions_mn_oc_tf = mn_oc_tf.predict_proba(tfidf_test)
predictions_mn_oc_count = mn_oc_count.predict_proba(count_test)

In [38]:
to_submission_csv(predictions_mn_oc_tf, 'submission_oc_mn_tf')
to_submission_csv(predictions_mn_oc_count, 'submission_oc_mn_count')

In [50]:
pd.DataFrame(
    data={'private': [0.84551, 0.82510], 'public': [0.85581, 0.83586]}, 
    index=['submission_oc_mn_count.csv', 'submission_oc_mn_tf.csv']
)

,private,public
submission_oc_mn_count.csv,0.84551,0.85581
submission_oc_mn_tf.csv,0.82510,0.83586


#### Hyperparameter Tuning

In [40]:
predictions_lr_count_tuned = np.zeros((len(test), len(classes)))
predictions_lr_tfidf_tuned = np.zeros((len(test), len(classes)))

In [41]:
estimator = OneVsRestClassifier(MultinomialNB ())

In [42]:
mn_oc_tf_tuned = GridSearchCV(estimator, parameters_mn_mo, n_jobs = -1, verbose = 10, scoring = 'accuracy')
mn_oc_tf_tuned.fit(tfidf_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(estimator=OneVsRestClassifier(estimator=MultinomialNB()),
             n_jobs=-1,
             param_grid=[{'estimator__alpha': [0.5, 0.6, 0.7, 0.8, 1.0],
                          'estimator__fit_prior': [True, False]}],
             scoring='accuracy', verbose=10)

In [43]:
predictions = mn_oc_tf_tuned.predict(tfidf_train)
print('TF-IDF Vectors: ', compute_accuracy(predictions, y_train), mn_oc_tf_tuned.best_params_)

TF-IDF Vectors:  toxic            93.609114
severe_toxic     98.989791
obscene          96.043141
threat           99.691673
insult           95.876444
identity_hate    99.106354
dtype: float64 {'estimator__alpha': 0.5, 'estimator__fit_prior': True}


In [44]:
predictions_mn_tfidf_tuned = mn_oc_tf_tuned.predict_proba(tfidf_test)
to_submission_csv(predictions_mn_tfidf_tuned, 'submission_oc_mn_tf_tuned')

In [45]:
mn_oc_count_tuned = GridSearchCV(estimator, parameters_mn_mo, n_jobs = -1, verbose = 10, scoring = 'accuracy')
mn_oc_count_tuned.fit(count_train, y_train)   

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(estimator=OneVsRestClassifier(estimator=MultinomialNB()),
             n_jobs=-1,
             param_grid=[{'estimator__alpha': [0.5, 0.6, 0.7, 0.8, 1.0],
                          'estimator__fit_prior': [True, False]}],
             scoring='accuracy', verbose=10)

In [46]:
predictions = mn_oc_count_tuned.predict(count_train)
print('Count Vectors: ', compute_accuracy(predictions, y_train), mn_oc_count_tuned.best_params_)

Count Vectors:  toxic            95.136961
severe_toxic     98.641984
obscene          96.708675
threat           99.555057
insult           96.463016
identity_hate    98.772333
dtype: float64 {'estimator__alpha': 1.0, 'estimator__fit_prior': True}


In [47]:
predictions_mn_count_tuned = mn_oc_count_tuned.predict_proba(count_test)
to_submission_csv(predictions_mn_count_tuned, 'submission_oc_mn_count_tuned')

In [49]:
pd.DataFrame(
    data={'private': [0.84551, 0.85045], 'public': [0.85581, 0.86105]}, 
    index=['submission_oc_mn_count_tuned.csv', 'submission_oc_mn_tf_tuned.csv']
)

,private,public
submission_oc_mn_count_tuned.csv,0.84551,0.85581
submission_oc_mn_tf_tuned.csv,0.85045,0.86105


### MultiOutput Classifier: Logistic Regression

#### Model Training

In [142]:
X_train = train ['comment_text']
X_test = test ['comment_text']
y_train = train.loc [ : , 'toxic' : ]
y_train

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [143]:
lr_mo_count = MultiOutputClassifier(LogisticRegression(class_weight = 'balanced', max_iter = 3000))
lr_mo_count.fit(count_train, y_train)

MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   max_iter=3000))

In [144]:
lr_mo_tf = MultiOutputClassifier(LogisticRegression(class_weight = 'balanced', max_iter = 3000))
lr_mo_tf.fit(tfidf_train, y_train)

MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   max_iter=3000))

In [146]:
predictions = lr_mo_tf.predict(tfidf_train)
print('TF-IDF Vectors: ' , compute_accuracy(predictions, y_train))

predictions = lr_mo_count.predict(count_train)
print('Count Vectors: ', compute_accuracy(predictions, y_train))

TF-IDF Vectors:  toxic            94.403118
severe_toxic     97.335982
obscene          97.324702
threat           99.067500
insult           95.850750
identity_hate    97.123537
dtype: float64
Count Vectors:  toxic            97.955142
severe_toxic     98.672064
obscene          98.804294
threat           99.741808
insult           97.874927
identity_hate    98.947177
dtype: float64


In [149]:
predictions_lr_mo_tf = lr_mo_tf.predict_proba(tfidf_test)
predictions_lr_mo_count = lr_mo_count.predict_proba(count_test)

In [150]:
to_submission_csv_multiclass(predictions_lr_mo_tf, 'submission_mo_lr_tf')
to_submission_csv_multiclass(predictions_lr_mo_count, 'submission_mo_lr_count')

In [154]:
pd.DataFrame(
    data={'private': [0.94036, 0.97063], 'public': [0.94400, 0.97183]}, 
    index=['submission_mo_lr_count.csv', 'submission_mo_lr_tf.csv']
)

,private,public
submission_mo_lr_count.csv,0.94036,0.94400
submission_mo_lr_tf.csv,0.97063,0.97183


#### Hyperparameter Tuning

In [152]:
predictions_lr_count_tuned = np.zeros((len(test), len(classes)))
predictions_lr_tfidf_tuned = np.zeros((len(test), len(classes)))

In [155]:
estimator = MultiOutputClassifier(LogisticRegression ())
lr_mo_tf_tuned = GridSearchCV(estimator, parameters_lr_mo, n_jobs = -1, verbose = 10, scoring = 'f1')
lr_mo_tf_tuned.fit(tfidf_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(estimator=MultiOutputClassifier(estimator=LogisticRegression()),
             n_jobs=-1,
             param_grid=[{'estimator__C': [1, 12, 15],
                          'estimator__class_weight': ['balanced', None],
                          'estimator__max_iter': [600, 1800, 3000]}],
             scoring='f1', verbose=10)

In [156]:
predictions = lr_mo_tf_tuned.predict(tfidf_train)
print('TF-IDF Vectors: ', compute_accuracy(predictions, y_train), lr_mo_tf_tuned.best_params_)

TF-IDF Vectors:  toxic            94.403118
severe_toxic     97.335982
obscene          97.324702
threat           99.067500
insult           95.850750
identity_hate    97.123537
dtype: float64 {'estimator__C': 1, 'estimator__class_weight': 'balanced', 'estimator__max_iter': 600}


In [157]:
predictions_lr_tfidf_tuned = lr_mo_tf_tuned.predict_proba(tfidf_test)
to_submission_csv_multiclass(predictions_lr_tfidf_tuned, 'submission_mo_lr_tf_tuned')

In [158]:
lr_mo_count_tuned = GridSearchCV(estimator, parameters_lr_mo, n_jobs = -1, verbose = 10, scoring = 'f1')
lr_mo_count_tuned.fit(count_train, y_train)   

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as show

GridSearchCV(estimator=MultiOutputClassifier(estimator=LogisticRegression()),
             n_jobs=-1,
             param_grid=[{'estimator__C': [1, 12, 15],
                          'estimator__class_weight': ['balanced', None],
                          'estimator__max_iter': [600, 1800, 3000]}],
             scoring='f1', verbose=10)

In [159]:
predictions = lr_mo_count_tuned.predict(count_train)
print('Count Vectors: ', compute_accuracy(predictions, y_train), lr_mo_count_tuned.best_params_)

Count Vectors:  toxic            97.936342
severe_toxic     98.640730
obscene          98.735359
threat           99.741808
insult           97.858633
identity_hate    98.945924
dtype: float64 {'estimator__C': 1, 'estimator__class_weight': 'balanced', 'estimator__max_iter': 600}


In [160]:
predictions_lr_count_tuned = lr_mo_count_tuned.predict_proba(count_test)
to_submission_csv_multiclass(predictions_lr_count_tuned, 'submission_mo_lr_count_tuned')

In [161]:
pd.DataFrame(
    data={'private': [0.93996, 0.97063], 'public': [0.94410, 0.97183]}, 
    index=['submission_mo_lr_count_tuned.csv', 'submission_mo_lr_tf_tuned.csv']
)

,private,public
submission_mo_lr_count_tuned.csv,0.93996,0.94410
submission_mo_lr_tf_tuned.csv,0.97063,0.97183


### MultiOutput Classifier: Multinomial Naive Bayes

#### Model Training

In [51]:
mn_mo_count = MultiOutputClassifier(MultinomialNB())
mn_mo_count.fit(count_train, y_train)

MultiOutputClassifier(estimator=MultinomialNB())

In [52]:
mn_mo_tf = MultiOutputClassifier(MultinomialNB())
mn_mo_tf.fit(tfidf_train, y_train)

MultiOutputClassifier(estimator=MultinomialNB())

In [53]:
predictions = mn_mo_tf.predict(tfidf_train)
print('TF-IDF Vectors: \n' , compute_accuracy(predictions, y_train))

predictions = mn_mo_count.predict(count_train)
print('Count Vectors: \n', compute_accuracy(predictions, y_train))

TF-IDF Vectors: 
 toxic            92.368287
severe_toxic     98.991045
obscene          95.384500
threat           99.697313
insult           95.356299
identity_hate    99.110741
dtype: float64
Count Vectors: 
 toxic            95.136961
severe_toxic     98.641984
obscene          96.708675
threat           99.555057
insult           96.463016
identity_hate    98.772333
dtype: float64


In [54]:
predictions_mnb_mo_tf = mn_mo_tf.predict_proba(tfidf_test)
predictions_mnb_mo_count = mn_mo_count.predict_proba(count_test)

In [55]:
to_submission_csv_multiclass(predictions_mnb_mo_tf, 'submission_mo_mn_tf')
to_submission_csv_multiclass(predictions_mnb_mo_count, 'submission_mo_mn_count')

In [56]:
pd.DataFrame(
    data={'private': [0.84551, 0.82510], 'public': [0.85581, 0.83586]}, 
    index=['submission_mo_mn_count.csv', 'submission_mo_mn_tf.csv']
)

,private,public
submission_mo_mn_count.csv,0.84551,0.85581
submission_mo_mn_tf.csv,0.82510,0.83586


#### Hyperparameter Tuning

In [38]:
predictions_mn_count_tuned = np.zeros((len(test), len(classes)))
predictions_mn_tfidf_tuned = np.zeros((len(test), len(classes)))

In [39]:
estimator = MultiOutputClassifier(MultinomialNB ())
mn_mo_tf_tuned = GridSearchCV(estimator, parameters_mn_mo, n_jobs = -1, verbose = 10, scoring = 'f1')
mn_mo_tf_tuned.fit(tfidf_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(estimator=MultiOutputClassifier(estimator=MultinomialNB()),
             n_jobs=-1,
             param_grid=[{'estimator__alpha': [0.5, 0.6, 0.7, 0.8, 1.0],
                          'estimator__fit_prior': [True, False]}],
             scoring='f1', verbose=10)

In [40]:
predictions = mn_mo_tf_tuned.predict(tfidf_train)
print('TF-IDF Vectors: \n', compute_accuracy(predictions, y_train), mn_mo_tf_tuned.best_params_)

TF-IDF Vectors:  toxic            93.609114
severe_toxic     98.989791
obscene          96.043141
threat           99.691673
insult           95.876444
identity_hate    99.106354
dtype: float64 {'estimator__alpha': 0.5, 'estimator__fit_prior': True}


In [41]:
predictions_mn_tfidf_tuned = mn_mo_tf_tuned.predict_proba(tfidf_test)
to_submission_csv_multiclass(predictions_mn_tfidf_tuned, 'submission_mo_mn_tf_tuned')

In [43]:
mn_mo_count_tuned = GridSearchCV(estimator, parameters_mn_mo, n_jobs = -1, verbose = 10, scoring = 'f1')
mn_mo_count_tuned.fit(count_train, y_train)   

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(estimator=MultiOutputClassifier(estimator=MultinomialNB()),
             n_jobs=-1,
             param_grid=[{'estimator__alpha': [0.5, 0.6, 0.7, 0.8, 1.0],
                          'estimator__fit_prior': [True, False]}],
             scoring='f1', verbose=10)

In [44]:
predictions = mn_mo_count_tuned.predict(count_train)
print('Count Vectors: \n', compute_accuracy(predictions, y_train), mn_mo_count_tuned.best_params_)

Count Vectors:  toxic            95.165162
severe_toxic     98.426406
obscene          96.575192
threat           99.472335
insult           96.317627
identity_hate    98.473407
dtype: float64 {'estimator__alpha': 0.5, 'estimator__fit_prior': True}


In [45]:
predictions_mn_count_tuned = mn_mo_count_tuned.predict_proba(count_test)
to_submission_csv_multiclass(predictions_mn_count_tuned, 'submission_mo_mn_count_tuned')

In [46]:
pd.DataFrame(
    data={'private': [0.87456, 0.85045], 'public': [0.88221, 0.86105]}, 
    index=['submission_mo_mn_count_tuned.csv', 'submission_mo_mn_tf_tuned.csv']
)

,private,public
submission_mo_mn_count_tuned.csv,0.87456,0.88221
submission_mo_mn_tf_tuned.csv,0.85045,0.86105


### Classifier Chain: Multinomial Naive Bayes

#### Model Training

In [37]:
mn_cc_tf = ClassifierChain(classifier = MultinomialNB(alpha = 1.0, fit_prior = True))
mn_cc_count = ClassifierChain(classifier = MultinomialNB(alpha = 1.0, fit_prior = True))

In [38]:
mn_cc_tf.fit(tfidf_train_5000, y_train)
mn_cc_count.fit(count_train_5000, y_train)

ClassifierChain(classifier=MultinomialNB(), require_dense=[True, True])

In [40]:
predictions_mn_cc_tf = mn_cc_tf.predict(tfidf_train_5000)
print('TF-IDF Vectors: \n' , compute_accuracy(predictions_mn_cc_tf.todense(), y_train))

predictions_mn_cc_count = mn_cc_count.predict(count_train_5000)
print('Count Vectors: \n', compute_accuracy(predictions_mn_cc_count.todense(), y_train))

TF-IDF Vectors: 
 toxic            95.029799
severe_toxic     98.715932
obscene          97.173672
threat           99.046819
insult           96.766330
identity_hate    95.881457
dtype: float64
Count Vectors: 
 toxic            94.139913
severe_toxic     97.492025
obscene          94.467666
threat           95.961672
insult           94.012070
identity_hate    93.246893
dtype: float64


In [41]:
predictions_mn_cc_tf = mn_cc_tf.predict_proba(tfidf_test_5000)
predictions_mn_cc_count = mn_cc_count.predict_proba(count_test_5000)

In [43]:
to_submission_csv(predictions_mn_cc_tf.todense(), 'submission_tfidf_mn_cc')
to_submission_csv(predictions_mn_cc_count.todense(), 'submission_count_mn_cc')

In [50]:
pd.DataFrame(
    data={'private': [0.92866, 0.94711], 'public': [0.92896, 0.94614]}, 
    index=['submission_count_mn_cc.csv', 'submission_tfidf_mn_cc.csv']
)

,private,public
submission_count_mn_cc.csv,0.92866,0.92896
submission_tfidf_mn_cc.csv,0.94711,0.94614


#### Hyperparameter Tuning

In [44]:
predictions_mn_count_tuned = np.zeros((len(test), len(classes)))
predictions_mn_tfidf_tuned = np.zeros((len(test), len(classes)))

In [ ]:
estimator = ClassifierChain(MultinomialNB ())
mn_cc_tf_tuned = GridSearchCV(estimator, parameters_mn_multi, n_jobs = -1, verbose = 10, scoring = 'f1')
mn_cc_tf_tuned.fit(tfidf_train_5000, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [ ]:
predictions = mn_cc_tf_tuned.predict(tfidf_train_5000)
print('TF-IDF Vectors: \n', compute_accuracy(predictions.todense(), y_train), mn_cc_tf_tuned.best_params_)

In [ ]:
predictions_mn_tfidf_tuned = mn_cc_tf_tuned.predict_proba(tfidf_test_5000)
to_submission_csv(predictions_mn_tfidf_tuned.todense(), 'submission_cc_mn_tf_tuned')

In [ ]:
mn_cc_count_tuned = GridSearchCV(estimator, parameters_mn_multi, n_jobs = -1, verbose = 10, scoring = 'f1')
mn_cc_count_tuned.fit(count_train_5000, y_train)

In [ ]:
predictions = mn_cc_count_tuned.predict(count_train_5000)
print('Count Vectors: \n', compute_accuracy(predictions.todense(), y_train), mn_cc_count_tuned.best_params_)

In [ ]:
predictions_mn_count_tuned = mn_cc_count_tuned.predict_proba(count_test_5000)
to_submission_csv(predictions_mn_count_tuned.todense(), 'submission_cc_mn_count_tuned')

In [ ]:
pd.DataFrame(
    data={'private': [0, 0], 'public': [0, 0]}, 
    index=['submission_cc_mn_count_tuned.csv', 'submission_cc_mn_tf_tuned.csv']
)

### Binary Relevance: Logistic Regression

#### Model Training

In [57]:
br_lr_tf = BinaryRelevance(classifier = LogisticRegression())
br_lr_count = BinaryRelevance(classifier = LogisticRegression())

In [ ]:
br_lr_tf.fit(tfidf_train_5000, y_train)
br_lr_count.fit(count_train_5000, y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

In [ ]:
predictions_br_lr_tf = br_lr_tf.predict(tfidf_train_5000)
print('TF-IDF Vectors: \n' , compute_accuracy(predictions_br_lr_tf.todense(), y_train))

predictions_br_lr_count = br_lr_count.predict(count_train_5000)
print('Count Vectors: \n', compute_accuracy(predictions_br_lr_count.todense(), y_train))

In [ ]:
predictions_br_lr_tf = br_lr_tf.predict_proba(tfidf_test_5000)
predictions_br_lr_count = br_lr_count.predict_proba(count_test_5000)

In [ ]:
to_submission_csv(predictions_br_lr_tf.todense(), 'submission_tfidf_lr_br')
to_submission_csv(predictions_br_lr_count.todense(), 'submission_count_lr_br')

In [ ]:
pd.DataFrame(
    data={'private': [0, 0], 'public': [0, 0]}, 
    index=['submission_count_lr_br.csv', 'submission_tfidf_lr_br.csv']
)

#### Hyperparameter Tuning

In [ ]:
predictions_lr_count_tuned = np.zeros((len(test), len(classes)))
predictions_lr_tfidf_tuned = np.zeros((len(test), len(classes)))

In [ ]:
estimator = BinaryRelevance(LogisticRegression ())
lr_cc_tf_tuned = GridSearchCV(estimator, parameters_lr_multi, n_jobs = -1, verbose = 10, scoring = 'f1')
lr_cc_tf_tuned.fit(tfidf_train_5000, y_train)

In [ ]:
predictions = lr_cc_tf_tuned.predict(tfidf_train_5000)
print('TF-IDF Vectors: \n', compute_accuracy(predictions.todense(), y_train), lr_cc_tf_tuned.best_params_)

In [ ]:
predictions_lr_tfidf_tuned = lr_cc_tf_tuned.predict_proba(tfidf_test_5000)
to_submission_csv(predictions_lr_tfidf_tuned.todense(), 'submission_lr_cc_tf_tuned')

In [ ]:
lr_br_count_tuned = GridSearchCV(estimator, parameters_lr_multi, n_jobs = -1, verbose = 10, scoring = 'f1')
lr_br_count_tuned.fit(count_train_5000, y_train)

In [ ]:
predictions = lr_br_count_tuned.predict(count_train_5000)
print('Count Vectors: \n', compute_accuracy(predictions.todense(), y_train), lr_br_count_tuned.best_params_)

In [ ]:
predictions_lr_count_tuned = lr_br_count_tuned.predict_proba(count_test_5000)
to_submission_csv(predictions_lr_count_tuned.todense(), 'submission_lr_cc_count_tuned')

In [ ]:
pd.DataFrame(
    data={'private': [0, 0], 'public': [0, 0]}, 
    index=['submission_count_lr_br.csv', 'submission_tfidf_lr_br.csv']
)

#### Model Selection

### Binary Relevance: Multinomial Naive Bayes

#### Model Training

In [ ]:
mn_br_count = MultiOutputClassifier(MultinomialNB())
mn_br_count.fit(tfidf_train_5000, y_train)

In [ ]:
mn_br_tf = MultiOutputClassifier(MultinomialNB())
mn_br_tf.fit(count_train_5000, y_train)

In [ ]:
predictions = mn_br_tf.predict(tfidf_train_5000)
print('TF-IDF Vectors: \n' , compute_accuracy(predictions.todense(), y_train))

predictions = mn_br_count.predict(count_train_5000)
print('Count Vectors: \n', compute_accuracy(predictions.todense(), y_train))

In [ ]:
predictions_mnb_br_tf = mn_br_tf.predict_proba(tfidf_test_5000)
predictions_mnb_br_count = mn_br_count.predict_proba(count_test_5000)

In [ ]:
to_submission_csv(predictions_mnb_br_tf.todense(), 'submission_br_mn_tf')
to_submission_csv(predictions_mnb_br_count.todense(), 'submission_br_mn_count')

In [ ]:
pd.DataFrame(
    data={'private': [0, 0], 'public': [0, 0]}, 
    index=['submission_br_mn_count.csv', 'submission_br_mn_tf.csv']
)

#### Hyperparameter Tuning

In [ ]:
predictions_mn_count_tuned = np.zeros((len(test), len(classes)))
predictions_mn_tfidf_tuned = np.zeros((len(test), len(classes)))

In [ ]:
estimator = ClassifierChain(MultinomialNB ())
mn_br_tf_tuned = GridSearchCV(estimator, parameters_mn_multi, n_jobs = -1, verbose = 10, scoring = 'f1')
mn_br_tf_tuned.fit(tfidf_train_5000, y_train)

In [ ]:
predictions = mn_br_tf_tuned.predict(tfidf_train_5000)
print('TF-IDF Vectors: \n', compute_accuracy(predictions.todense(), y_train), mn_br_tf_tuned.best_params_)

In [ ]:
predictions_mn_tfidf_tuned = mn_br_tf_tuned.predict_proba(tfidf_test_5000)
to_submission_csv(predictions_mn_tfidf_tuned.todense(), 'submission_br_mn_tf_tuned')

In [ ]:
mn_br_count_tuned = GridSearchCV(estimator, parameters_mn_multi, n_jobs = -1, verbose = 10, scoring = 'f1')
mn_br_count_tuned.fit(count_train_5000, y_train)   

In [ ]:
predictions = mn_br_count_tuned.predict(count_train_5000)
print('Count Vectors: \n', compute_accuracy(predictions.todense(), y_train), mn_br_count_tuned.best_params_)

In [ ]:
predictions_mn_count_tuned = mn_br_count_tuned.predict_proba(count_test_5000)
to_submission_csv(predictions_mn_count_tuned.todense(), 'submission_br_mn_count_tuned')

In [ ]:
pd.DataFrame(
    data={'private': [0, 0], 'public': [0, 0]}, 
    index=['submission_br_mn_count_tuned.csv', 'submission_br_mn_tf_tuned.csv']
)

#### Model Selection

### Multinomial Naive Bayes using TF-IDF Vectorizer

#### Model Training

In [ ]:
X_train = train ['comment_text']
X_test = test ['comment_text']

In [ ]:
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

In [ ]:
tfidf_test = tfidf_vectorizer.transform(X_test)

In [ ]:
arr_model = []
counter = 0
for class_ in classes:
    y_train = train[class_]
    model = MultinomialNB ()
    model.fit(tfidf_train, y_train)
    predictions = model.predict(tfidf_train)
    arr_model.append(model)
    counter = counter + 1

In [ ]:
counter = 0
for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    predictions = arr_model [counter].predict(tf_idf_train)
    print(compute_accuracy(predictions, y_train))
    counter = counter + 1

In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission ['id'] = test ['id'] 

counter = 0
for class_ in classes:
    predictions = arr_model [counter].predict(tf_idf_test)
    sample_submission [class_] = predictions
    counter = counter + 1
    
sample_submission.to_csv(f'results/submission_tfidf_nb.csv', index = False) 

#### Hyperparameter Tuning

In [ ]:
X = train ['comment_text']

In [ ]:
final_hyperparameters = []
classes = train.columns [2:]
arr_model = []
counter = 0

for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    best_score = 0
    
    model = MultinomialNB ()
    
    X_train, X_val, y_train, y_val = train_test_split (X, y_train, test_size = 0.25, stratify = y_train)
    
    X_train_sparse_matrix = tf_idf_vectorizer.fit_transform(X_train)
    X_validation_sparse_matrix = tf_idf_vectorizer.transform(X_val)
    
    for g in ParameterGrid(parameters_mnb):

        model.set_params(**g)

        model.fit(X_train_sparse_matrix, y_train)
        predictions = model.predict (X_train_sparse_matrix)
        train_acc = compute_accuracy (predictions, y_train)

        predictions = model.predict (X_validation_sparse_matrix)
        val_acc = compute_accuracy (predictions, y_val)

        if val_acc > best_score:
            best_score = val_acc
            best_grid = g
    
    print("Best accuracy: ", best_score, "%")
    print("Best grid: ", best_grid)
    temp = mn_hyper_parameter (class_, best_grid['alpha'], best_grid['fit_prior'])
    final_hyperparameters.append(temp)

#### Model Selection

In [ ]:
X_train = train ['comment_text']
X_test = test ['comment_text']

In [ ]:
tf_idf_train = tf_idf_vectorizer.fit_transform(X_train)
tf_idf_test = tf_idf_vectorizer.transform(X_test)

In [ ]:
classes = train.columns [2:]
arr_model = []
counter = 0
for class_ in classes:
    print("Class: ", class_)
    
    y_train = train[class_]
    temp = final_hyperparameters [counter]
    model = MultinomialNB (alpha = temp.alpha, fit_prior = temp.fit_prior)

    model.fit(tf_idf_train, y_train)
    predictions = model.predict(tf_idf_train)
    print(compute_accuracy(predictions, y_train))
    
    arr_model.append(model)
    counter = counter + 1

In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission ['id'] = test ['id'] 
counter = 0

for class_ in classes:
    predictions = arr_model [counter].predict(tf_idf_test)
    sample_submission [class_] = predictions
    counter = counter + 1
    
sample_submission.to_csv(f'results/submission_tf_idf_mn_tuned.csv', index = False) 

### Multinomial Naive Bayes using Count Vectorizer

#### Model Training

In [ ]:
X_train = train ['comment_text']
X_test = test ['comment_text']

In [ ]:
count_train = count_vectorizer.fit_transform(X_train)

In [ ]:
count_test = count_vectorizer.transform(X_test)

In [ ]:
classes = train.columns [2:]
arr_model = []
counter = 0
for class_ in classes:
    y_train = train[class_]
    model = MultinomialNB ()
    model.fit(count_train, y_train)
    
    predictions = model.predict(count_train)
    arr_model.append(model)
    counter = counter + 1

In [ ]:
counter = 0
for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    predictions = arr_model [counter].predict(count_train)
    print(compute_accuracy(predictions, y_train))
    counter = counter + 1

In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')

sample_submission ['id'] = test ['id'] 
counter = 0
for class_ in classes:
    predictions = arr_model [counter].predict(count_test)
    sample_submission [class_] = predictions
    counter = counter + 1
sample_submission.to_csv(f'results/submission_count_nb.csv', index = False) 

#### Hyperparameter Tuning

In [ ]:
X = train ['comment_text']

In [ ]:
final_hyperparameters = []
classes = train.columns [2:]
arr_model = []
counter = 0

for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    best_score = 0
    
    model = MultinomialNB ()
    
    X_train, X_val, y_train, y_val = train_test_split (X, y_train, test_size = 0.25, stratify = y_train)
    
    X_train_sparse_matrix = count_vectorizer.fit_transform(X_train)
    X_validation_sparse_matrix = count_vectorizer.transform(X_val)
    
    for g in ParameterGrid(parameters_mnb):

        model.set_params(**g)

        model.fit(X_train_sparse_matrix, y_train)
        predictions = model.predict (X_train_sparse_matrix)
        train_acc = compute_accuracy (predictions, y_train)

        predictions = model.predict (X_validation_sparse_matrix)
        val_acc = compute_accuracy (predictions, y_val)

        if val_acc > best_score:
            best_score = val_acc
            best_grid = g
    
    print("Best accuracy: ", best_score, "%")
    print("Best grid: ", best_grid)
    temp = mn_hyper_parameter (class_, best_grid['alpha'], best_grid['fit_prior'])
    final_hyperparameters.append(temp)

#### Model Selection

In [ ]:
X_train = train ['comment_text']
X_test = test ['comment_text']

In [ ]:
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

In [ ]:
classes = train.columns [2:]
arr_model = []
counter = 0
for class_ in classes:
    print("Class: ", class_)
    
    y_train = train[class_]
    temp = final_hyperparameters [counter]
    model = MultinomialNB (alpha = temp.alpha, fit_prior = temp.fit_prior)

    model.fit(count_train, y_train)
    predictions = model.predict(count_train)
    print(compute_accuracy(predictions, y_train))
    
    arr_model.append(model)
    counter = counter + 1

In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission ['id'] = test ['id'] 
counter = 0

for class_ in classes:
    predictions = arr_model [counter].predict(count_test)
    sample_submission [class_] = predictions
    counter = counter + 1
    
sample_submission.to_csv(f'results/submission_count_mn_tuned.csv', index = False) 

### Logistic Regression using TF-IDF Vectorizer

#### Model Training

In [ ]:
X_train = train ['comment_text']
X_test = test ['comment_text']

In [ ]:
tf_idf_train = tf_idf_vectorizer.fit_transform(X_train)

In [ ]:
tf_idf_test = tf_idf_vectorizer.transform(X_test)

In [ ]:
classes = train.columns [2:]
arr_model = []
counter = 0
for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    model = LogisticRegression (n_jobs=-1)

    model.fit(tf_idf_train, y_train)
    predictions = model.predict(tf_idf_train)
    print(compute_accuracy(predictions, y_train))
    arr_model.append(model)
    counter = counter + 1

In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission ['id'] = test ['id'] 
counter = 0

for class_ in classes:
    predictions = arr_model [counter].predict_proba(tf_idf_test)[:,1]
    sample_submission [class_] = predictions
    counter = counter + 1
    
sample_submission.to_csv(f'results/submission_logreg_1.csv', index = False) 

#### Hyperparameter Tuning

In [ ]:
X = train ['comment_text']

In [ ]:
final_hyperparameters = []
classes = train.columns [2:]
arr_model = []
counter = 0

for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    best_score = 0
    
    model = LogisticRegression ()
    
    X_train, X_val, y_train, y_val = train_test_split (X, y_train, test_size = 0.25, stratify = y_train)
    
    X_train_sparse_matrix = tf_idf_vectorizer.fit_transform(X_train)
    X_validation_sparse_matrix = tf_idf_vectorizer.transform(X_val)
    
    for g in ParameterGrid(parameters_lr):

        model.set_params(**g)

        model.fit(X_train_sparse_matrix, y_train)
        predictions = model.predict (X_train_sparse_matrix)
        train_acc = compute_accuracy (predictions, y_train)

        predictions = model.predict (X_validation_sparse_matrix)
        val_acc = compute_accuracy (predictions, y_val)

        if val_acc > best_score:
            best_score = val_acc
            best_grid = g
    
    print("Best accuracy: ", best_score, "%")
    print("Best grid: ", best_grid)
    temp = lr_hyperparameter (class_, best_grid['C'], best_grid['max_iter'])
    final_hyperparameters.append(temp)

#### Model Selection

In [ ]:
X_train = train ['comment_text']
X_test = test ['comment_text']

In [ ]:
tf_idf_train = tf_idf_vectorizer.fit_transform(X_train)
tf_idf_test = tf_idf_vectorizer.transform(X_test)

In [ ]:
classes = train.columns [2:]
arr_model = []
counter = 0
for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    temp = final_hyperparameters [counter]
    model = LogisticRegression (C = temp.c, max_iter = temp.max_iter)

    model.fit(tf_idf_train, y_train)
    predictions = model.predict(tf_idf_train)
    print(compute_accuracy(predictions, y_train))
    arr_model.append(model)
    counter = counter + 1

In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission ['id'] = test ['id'] 
counter = 0

for class_ in classes:
    predictions = arr_model [counter].predict(tf_idf_test)
    sample_submission [class_] = predictions
    counter = counter + 1
    
sample_submission.to_csv(f'results/submission_tf_idf_log_reg_tuned.csv', index = False) 

### Logistic Regression using Count Vectorizer

#### Model Training

In [ ]:
X_train = train ['comment_text']
X_test = test ['comment_text']

In [ ]:
count_train = count_vectorizer.fit_transform(X_train)

In [ ]:
count_test = count_vectorizer.transform(X_test)

In [ ]:
classes = train.columns [2:]
arr_model = []
counter = 0
for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    model = LogisticRegression ()

    model.fit(count_train, y_train)
    predictions = model.predict(count_train)
    print(compute_accuracy(predictions, y_train))
    arr_model.append(model)
    counter = counter + 1

In [ ]:
counter = 0
for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    predictions = arr_model [counter].predict(count_train)
    print(compute_accuracy(predictions, y_train))
    counter = counter + 1

In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission ['id'] = test ['id'] 
counter = 0

for class_ in classes:
    predictions = arr_model [counter].predict(count_test)
    sample_submission [class_] = predictions
    counter = counter + 1
    
sample_submission.to_csv(f'results/submission_count_log_reg.csv', index = False) 

#### Hyperparameter Tuning

In [ ]:
X = train ['comment_text']

In [ ]:
final_hyperparameters = []
classes = train.columns [2:]
arr_model = []
counter = 0

for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    best_score = 0
    
    model = LogisticRegression ()
    
    X_train, X_val, y_train, y_val = train_test_split (X, y_train, test_size = 0.25, stratify = y_train)
    
    X_train_sparse_matrix = count_vectorizer.fit_transform(X_train)
    X_validation_sparse_matrix = count_vectorizer.transform(X_val)
    
    for g in ParameterGrid(parameters_lr):

        model.set_params(**g)

        model.fit(X_train_sparse_matrix, y_train)
        predictions = model.predict (X_train_sparse_matrix)
        train_acc = compute_accuracy (predictions, y_train)

        predictions = model.predict (X_validation_sparse_matrix)
        val_acc = compute_accuracy (predictions, y_val)

        if val_acc > best_score:
            best_score = val_acc
            best_grid = g
    
    print("Best accuracy: ", best_score, "%")
    print("Best grid: ", best_grid)
    temp = lr_hyperparameter (class_, best_grid['C'], best_grid['max_iter'])
    final_hyperparameters.append(temp)

#### Model Selection

In [ ]:
X_train = train ['comment_text']
X_test = test ['comment_text']

In [ ]:
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

In [ ]:
classes = train.columns [2:]
arr_model = []
counter = 0
for class_ in classes:
    print("Class: ", class_)
    y_train = train[class_]
    temp = final_hyperparameters [counter]
    model = LogisticRegression (C = temp.c, max_iter = temp.max_iter)

    model.fit(count_train, y_train)
    predictions = model.predict(count_train)
    print(compute_accuracy(predictions, y_train))
    arr_model.append(model)
    counter = counter + 1

In [ ]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission ['id'] = test ['id'] 
counter = 0

for class_ in classes:
    predictions = arr_model [counter].predict(count_test)
    sample_submission [class_] = predictions
    counter = counter + 1
    
sample_submission.to_csv(f'results/submission_count_log_reg_tuned.csv', index = False) 